In [1]:
train = pd.read_csv("dataset/train.csv")

NameError: name 'pd' is not defined

In [2]:
root_classes_folder = "dataset/trainclasses"
dataset = Dataset("dataset")
sub_dataset_train_path = "/".join([root_classes_folder, "train"])
sub_dataset_dev_path = "/".join([root_classes_folder, "dev"])
files_number = len(dataset_train_label)
print("Sorting", files_number, "images into", root_classes_folder)

NameError: name 'Dataset' is not defined

In [2]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


import random

from keras.preprocessing import image
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import SGD, RMSprop
from keras import backend as K
import pandas as pd

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [4]:
y_train[0]

NameError: name 'y_train' is not defined

In [5]:
train.head(2)

NameError: name 'train' is not defined

In [2]:
img = image.load_img("./dataset/train/0000e88ab.jpg", target_size=(200, 100, 3))
x = image.img_to_array(img)
img1 = image.load_img("./dataset/train/0000e88ab.jpg", target_size=(200, 100, 3))
y = image.img_to_array(img1)

In [7]:
a = np.concatenate([[x],[y]], axis = 0)

In [8]:
a.shape

(2, 200, 100, 3)

In [9]:
a = np.array([[[1, 2], [3, 4]]])
b = np.array([[[5, 6], [7, 8]]])
np.vstack([a,b])

array([[[1, 2],
        [3, 4]],

       [[5, 6],
        [7, 8]]])

In [10]:
train_data = pd.read_csv("dataset/train.csv")

In [76]:
import time
time1 = time.time()
df_train.set_index('Id', inplace=True)
df_train.sort_index(inplace=True)
df_train.head(30)

for i in range(5000):
    df_train.loc[df_train.index=='new_whale']
    
print(time.time()-time1)

11.543234825134277


In [71]:
#df_train.set_index('Id', inplace=True)
time1 = time.time()
df_train.head(30)

for i in range(5000):
    df_train.loc[df_train.Id=='new_whale']
print(time.time()-time1)

14.283166408538818


In [28]:
import time
RANDOM_SEED = 1475

train_data = pd.read_csv("dataset/train.csv")
time1 = time.time()
train_data.set_index('Id', inplace=True)
train_data.sort_index(inplace=True)
fake_data = []
real_data = []
k=0

for sub_index in set(train_data.index):
    if sub_index!="new_whale":
        print(k)
        print(sub_index)
        k+=1
        images_same_class = list(set(train_data.loc[train_data.index==sub_index].Image))
        images_different_class = list(set(train_data.loc[train_data.index!=sub_index].Image))

        for i in range(len(images_same_class)):
            for j in range(i, len(images_same_class)):
                real_data.append(np.array([images_same_class[i], images_same_class[j],1]))
                fake_data.append(np.array([images_same_class[i], images_different_class[np.random.randint(len(images_different_class))],0]))
                fake_data.append(np.array([images_same_class[i], images_different_class[np.random.randint(len(images_different_class))],0]))

fake_data = np.vstack(fake_data)
real_data = np.vstack(real_data)
print((time.time()-time1)/60)

            
        

0
w_05bf34e
1
w_3e4a1a5
2
w_c94f151
3
w_5a49beb
4
w_0a4f5c9
5
w_10a4e10
6
w_2f702db
7
w_6cf6bf4
8
w_3e4b155
9
w_8ab401c
10
w_a0534e5
11
w_aeabde9
12
w_551f361
13
w_4e88955
14
w_88ff97d
15
w_91a2a20
16
w_d9aecac
17
w_27be996
18
w_8936c44
19
w_d5e0aba
20
w_760c469
21
w_3b5aee2
22
w_fa7cd91
23
w_9037ac5
24
w_989e081
25
w_5ce4848
26
w_88503f1
27
w_e6325d7
28
w_037b23f
29
w_e2a09d4
30
w_936aa13
31
w_090c801
32
w_ecad2c9
33
w_29c88c2
34
w_6af9b02
35
w_997cda5
36
w_e93b569
37
w_af294f7
38
w_c6c89db
39
w_04c9b23
40
w_49ee8e0
41
w_925332d
42
w_0655608
43
w_2f8895a
44
w_88065f2
45
w_a33ed90
46
w_540ac0a
47
w_6dedb5c
48
w_3853f4b
49
w_80b474e
50
w_bc57d81
51
w_2c2e6b9
52
w_7a61cb2
53
w_7b05eeb
54
w_776a885
55
w_b72cdc1
56
w_c2426a1
57
w_d215a68
58
w_7085ad6
59
w_df6efd0
60
w_d8ae71c
61
w_c405fe4
62
w_61e4248
63
w_be1a38b
64
w_49ce813
65
w_fe0c54f
66
w_a08f3cd
67
w_b51e6f6
68
w_2f3badb
69
w_8d9af77
70
w_accb0cb
71
w_d06e2d0
72
w_ede6c73
73
w_c6c60c8
74
w_4c2af2c
75
w_b421d21
76
w_71d5ba2
77
w_7d28

597
w_a386d0b
598
w_4c18cce
599
w_f28a7a2
600
w_8da30ad
601
w_dccbb2e
602
w_825910b
603
w_d6907f5
604
w_dc27ce9
605
w_9b16b48
606
w_08753b7
607
w_dc94d80
608
w_c465f0b
609
w_3ea05d5
610
w_7e9ae57
611
w_e6ff725
612
w_ff65f67
613
w_1afed1e
614
w_9fecf9d
615
w_3789389
616
w_1fc42e2
617
w_d0475b2
618
w_7afb40a
619
w_37e61c2
620
w_d6553ba
621
w_58afeb2
622
w_0e34e76
623
w_0e9f07a
624
w_647da1f
625
w_e0b6a42
626
w_33fccbe
627
w_a6de601
628
w_b4e9da5
629
w_e56d7b2
630
w_e97509e
631
w_f8511e9
632
w_0fc3e41
633
w_48619b3
634
w_4b4f9b7
635
w_5510cf3
636
w_b4b2b67
637
w_a8a6e1f
638
w_2694603
639
w_363a5aa
640
w_8279f22
641
w_009c9c5
642
w_94ea240
643
w_14e1334
644
w_455bc02
645
w_548a268
646
w_588f349
647
w_90336d0
648
w_ec64e27
649
w_682897f
650
w_9eb9853
651
w_fb3959d
652
w_0655a0d
653
w_6cb0480
654
w_d09a047
655
w_5c4c166
656
w_614b549
657
w_f035513
658
w_55e0064
659
w_d602efb
660
w_5833b3d
661
w_a3bb16b
662
w_0630c3e
663
w_1afff3f
664
w_fcf1d68
665
w_1c55cc3
666
w_9ba4a9a
667
w_f128a4d
668
w_

1176
w_3108dce
1177
w_c616473
1178
w_fd906cc
1179
w_febd4fe
1180
w_325f8d2
1181
w_7206ab2
1182
w_cad4b47
1183
w_7957b08
1184
w_49cd60d
1185
w_861e504
1186
w_aa47677
1187
w_a8b34a0
1188
w_ae8982d
1189
w_091ec23
1190
w_b860053
1191
w_3334d3f
1192
w_143ea92
1193
w_99d1f45
1194
w_7087c88
1195
w_51f03a3
1196
w_bf46204
1197
w_f382f59
1198
w_056710d
1199
w_faf5287
1200
w_06150e1
1201
w_122eec1
1202
w_633ead1
1203
w_aaf3463
1204
w_093d284
1205
w_c1715f5
1206
w_e7f704a
1207
w_9eb6492
1208
w_f3d2e31
1209
w_46ba1ce
1210
w_faac069
1211
w_f84e6e1
1212
w_a5f74b2
1213
w_26923f7
1214
w_5ad70f3
1215
w_a71d6bf
1216
w_a780b2d
1217
w_7fe4082
1218
w_b1abd47
1219
w_dfc9865
1220
w_e3fb839
1221
w_6d527ff
1222
w_22cdc5f
1223
w_4953900
1224
w_c2749eb
1225
w_bece5a4
1226
w_cbdae16
1227
w_8cee3d3
1228
w_0ffa15f
1229
w_da15b9b
1230
w_96985c1
1231
w_463f219
1232
w_d71a790
1233
w_07e0c3c
1234
w_aebdb5f
1235
w_acfc8c1
1236
w_10f67bb
1237
w_c734fa4
1238
w_9d86594
1239
w_f01ade6
1240
w_11ce746
1241
w_7fcf284
1242
w_a70

1724
w_1f1774e
1725
w_10eece8
1726
w_717756b
1727
w_dc14a8e
1728
w_799b208
1729
w_02469a1
1730
w_0a8d318
1731
w_3180b48
1732
w_6f3a3ec
1733
w_a88c767
1734
w_343f088
1735
w_1531bf5
1736
w_027f528
1737
w_8620eca
1738
w_d063697
1739
w_3ffae11
1740
w_bdfd856
1741
w_e9ef5e9
1742
w_edce644
1743
w_f05a1df
1744
w_c0f1e04
1745
w_0bfc890
1746
w_1106cbb
1747
w_84f9ab0
1748
w_78e7020
1749
w_57acd97
1750
w_9ca67be
1751
w_e732406
1752
w_954fec8
1753
w_c057db0
1754
w_353d249
1755
w_7b098ce
1756
w_7ffc63c
1757
w_935a516
1758
w_c720f04
1759
w_44fc980
1760
w_f5aadde
1761
w_9748a78
1762
w_3040ecd
1763
w_71fa596
1764
w_7330627
1765
w_4e63f7a
1766
w_6d8b67b
1767
w_1b0c4b7
1768
w_384e3b2
1769
w_982f0b3
1770
w_b82d0eb
1771
w_ee2326a
1772
w_0bfdcca
1773
w_5c083e6
1774
w_0729126
1775
w_0c76e03
1776
w_b2af5eb
1777
w_ba01717
1778
w_f4980e2
1779
w_99d33cf
1780
w_aec2b6d
1781
w_7efe670
1782
w_cdb8e7f
1783
w_c0c5cae
1784
w_2c3c217
1785
w_f4a9357
1786
w_77150d0
1787
w_d7ef410
1788
w_6c33987
1789
w_8d76b75
1790
w_cbd

w_7c26ef7
2273
w_1faf1e4
2274
w_f39c242
2275
w_8d52508
2276
w_ec7e60a
2277
w_c97ad94
2278
w_e129173
2279
w_90df70c
2280
w_db2d591
2281
w_2f94559
2282
w_3c4b1d0
2283
w_15fb899
2284
w_deda5f7
2285
w_8b83220
2286
w_8dda2f6
2287
w_8debfa9
2288
w_05d52d0
2289
w_be6e58a
2290
w_470be90
2291
w_9ba6731
2292
w_8fad4d2
2293
w_9cff87c
2294
w_8a9e295
2295
w_d236b51
2296
w_06fb64a
2297
w_f2f7550
2298
w_0ef790d
2299
w_d89acf0
2300
w_bd9f354
2301
w_20edfc3
2302
w_3bdf2c6
2303
w_68a89b5
2304
w_ab8af2b
2305
w_ef83760
2306
w_745ee82
2307
w_2980f85
2308
w_15bf3e3
2309
w_5045d2c
2310
w_1423c1f
2311
w_5529f3b
2312
w_57a546c
2313
w_81eee24
2314
w_335bdd1
2315
w_84dc8cd
2316
w_b7cf3f5
2317
w_be235cb
2318
w_11ff425
2319
w_df5bf3d
2320
w_e0a42ad
2321
w_76e67dc
2322
w_3002b59
2323
w_12fe3a8
2324
w_b19f2f9
2325
w_b7babc0
2326
w_9d49420
2327
w_05fbf14
2328
w_7a8ce16
2329
w_914b110
2330
w_fe59266
2331
w_c481650
2332
w_329c19c
2333
w_e1702d4
2334
w_245ac74
2335
w_ef00141
2336
w_317ae6c
2337
w_910831a
2338
w_ecd9cc4


2825
w_83d10c1
2826
w_d8cb231
2827
w_f0663e5
2828
w_59fb9d2
2829
w_9732911
2830
w_0714a46
2831
w_9f02ca1
2832
w_a331c8b
2833
w_2efa6ab
2834
w_016f0ea
2835
w_68ed1f9
2836
w_9b6d9b7
2837
w_bfae26b
2838
w_cf1e889
2839
w_d4c30a9
2840
w_e107383
2841
w_0c7ab0d
2842
w_bdf0447
2843
w_ea6a9f4
2844
w_3bfa4f8
2845
w_d962e2b
2846
w_3af5f93
2847
w_7709151
2848
w_abf0b13
2849
w_1920193
2850
w_e65664f
2851
w_7d34a32
2852
w_bbe79c3
2853
w_10bb1ba
2854
w_98d094e
2855
w_2c0f96c
2856
w_a8b65e2
2857
w_45b1b01
2858
w_b2606f3
2859
w_2112103
2860
w_efa100e
2861
w_35863c5
2862
w_692734c
2863
w_90c9763
2864
w_3d7bc85
2865
w_538259a
2866
w_59088ff
2867
w_ef30316
2868
w_f78a4b1
2869
w_32b01d7
2870
w_2ed1e39
2871
w_0323574
2872
w_8b943ef
2873
w_69a58b2
2874
w_b8f1666
2875
w_bcb2fe3
2876
w_c028816
2877
w_0140c92
2878
w_c99807e
2879
w_832917d
2880
w_2fc317c
2881
w_f449c3d
2882
w_8d80478
2883
w_3a1e62f
2884
w_0b4f7cf
2885
w_9078d11
2886
w_8cce8ce
2887
w_c0e2505
2888
w_1289b63
2889
w_d0589c5
2890
w_32de0c5
2891
w_100

w_0003639
3376
w_0bc1de9
3377
w_1f1cee1
3378
w_6fb7fe9
3379
w_2de1c5c
3380
w_9888ce0
3381
w_d3e3b56
3382
w_8c8c45a
3383
w_bdb3a05
3384
w_c88caf7
3385
w_2284d86
3386
w_c178056
3387
w_bd73f1d
3388
w_853aa18
3389
w_45e0e10
3390
w_db96784
3391
w_64055b4
3392
w_8261f4d
3393
w_f314516
3394
w_1344da3
3395
w_1247b8c
3396
w_562b227
3397
w_8092ea7
3398
w_550ca18
3399
w_2af46a8
3400
w_44803e4
3401
w_81d807a
3402
w_a33b6d6
3403
w_5645ed1
3404
w_c56fefc
3405
w_b403feb
3406
w_b421a6a
3407
w_e3e617e
3408
w_72f3685
3409
w_9be951c
3410
w_fd4376a
3411
w_6e0e2ae
3412
w_4735be5
3413
w_c379d78
3414
w_8dddbee
3415
w_9297b97
3416
w_2807565
3417
w_0350bf4
3418
w_0e93f25
3419
w_6d1f159
3420
w_0a0b4c6
3421
w_6840783
3422
w_b1461a9
3423
w_f8746c3
3424
w_303a3c4
3425
w_689e1cd
3426
w_6dbfd7a
3427
w_13937e4
3428
w_8b61e65
3429
w_adedd1f
3430
w_119f570
3431
w_8d09425
3432
w_df34804
3433
w_88b3466
3434
w_3a1969c
3435
w_704a8a1
3436
w_f6524e1
3437
w_420254e
3438
w_84c23bb
3439
w_3cde3ea
3440
w_7f5d9fd
3441
w_df32eec


3924
w_655fe84
3925
w_7d8c37c
3926
w_b012783
3927
w_ac5aa34
3928
w_ad343a6
3929
w_5d687a9
3930
w_b79abd4
3931
w_b3cfa1d
3932
w_a123aa1
3933
w_5d36478
3934
w_e89442d
3935
w_af5d5e3
3936
w_e05c1a2
3937
w_aae50c8
3938
w_7f4e251
3939
w_b938e96
3940
w_c148ab4
3941
w_7d296e9
3942
w_eced610
3943
w_0ecda0a
3944
w_6c4a986
3945
w_c73e8e9
3946
w_0dc84fd
3947
w_1ea8997
3948
w_d5bbfbc
3949
w_f0fe284
3950
w_f87bc93
3951
w_06460d7
3952
w_8c959a6
3953
w_d83d524
3954
w_ee318d3
3955
w_dec7ffd
3956
w_3bf887a
3957
w_ff5fc91
3958
w_81ed5f8
3959
w_0adb403
3960
w_302799d
3961
w_11bd416
3962
w_567d88c
3963
w_905944f
3964
w_c98aaad
3965
w_0d43823
3966
w_1f7e291
3967
w_a9a3878
3968
w_315c854
3969
w_6658081
3970
w_721e78f
3971
w_1fe6c6e
3972
w_1a31542
3973
w_2f79ae3
3974
w_e4ed810
3975
w_3ef0017
3976
w_8b8dcf5
3977
w_bb4f52e
3978
w_f0708c8
3979
w_0b10e7c
3980
w_2d5b0e1
3981
w_6ebfe7b
3982
w_0a8e080
3983
w_118fad5
3984
w_10f3b75
3985
w_0dfb4f6
3986
w_6755824
3987
w_a26cf66
3988
w_55ef07b
3989
w_2ef6741
3990
w_b43

4473
w_778e474
4474
w_8a56270
4475
w_db86325
4476
w_f86b16b
4477
w_0749e87
4478
w_b747aeb
4479
w_099bcbc
4480
w_cfe7928
4481
w_25d662d
4482
w_8fa8bef
4483
w_9b46d56
4484
w_76d1f84
4485
w_abd456f
4486
w_ed620af
4487
w_71dfca3
4488
w_7cd0b86
4489
w_4f717ba
4490
w_64062df
4491
w_0fe07b7
4492
w_c875e45
4493
w_030e2cf
4494
w_d947e85
4495
w_51c003e
4496
w_82d50c5
4497
w_76b8e6f
4498
w_263bd97
4499
w_244ab03
4500
w_1d6a9f7
4501
w_179c9f0
4502
w_83a4279
4503
w_d9405b3
4504
w_88b2b19
4505
w_ed009d3
4506
w_5cc8711
4507
w_6f62459
4508
w_34053b9
4509
w_dc7d6d7
4510
w_e5ad760
4511
w_7ec7c3b
4512
w_615a631
4513
w_99e43ae
4514
w_0a0c768
4515
w_283ceeb
4516
w_b2ef717
4517
w_b433be9
4518
w_b72dc3d
4519
w_4888003
4520
w_a87ec7e
4521
w_01d790e
4522
w_cfdd1c0
4523
w_d12f2a0
4524
w_7064855
4525
w_e36fb17
4526
w_888d8e3
4527
w_e8e08e6
4528
w_583aba1
4529
w_fb52b93
4530
w_90a7b56
4531
w_0577ccc
4532
w_effbe89
4533
w_0f84e7c
4534
w_c5e1472
4535
w_9c2f8e2
4536
w_0ce537a
4537
w_175d6fa
4538
w_3250a08
4539
w_a27

In [ ]:
total_data = np.concatenate([fake_data, real_data])
total_data.shape

In [25]:
np.ones((171268,1))

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

array([['ea8520852.jpg', '4a04cdc8a.jpg'],
       ['ea8520852.jpg', '15b58a61b.jpg'],
       ['ea8520852.jpg', '37f214f08.jpg'],
       ...,
       ['1.0', '1.0'],
       ['1.0', '1.0'],
       ['1.0', '1.0']], dtype='<U32')

In [4]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import SGD, RMSprop
from keras import backend as K

class Dataset:
    """
    Manage dataset loading
    
    :param dataset_path: str, path to the dataset folder
    """
    
    def __init__(self, dataset_path):
        
        # Build and store dataset paths
        self.dataset_path_train = dataset_path + "/train"
        self.dataset_path_test = dataset_path + "/test"
        
        # Build train.csv path
        self.dataset_path_train_label = dataset_path + "/train.csv"
        
        # Generate pandas dataframe of whales id <-> file matching
        self.dataset_train_label = self.get_train_label()
        
        ## Get pictures paths
#         self.les_im_path_train = self.get_im_path(self.dataset_path_train + "/*.jpg")
#         self.les_im_path_test = self.get_im_path(self.dataset_path_test + "/*.jpg")
        
    def get_im_path(self, dataset_path):
        """
        Get pictures path under the given folder path
        :param dataset_path: str, path to the dataset folder
        
        :output les_im_path: list of string, .jpg picture paths under the dataset_path folder
        """
        print("Getting images path from", dataset_path)
        les_im_path = glob.glob(dataset_path)
        les_im_path.sort()
        return les_im_path
    
    def get_train_label(self):
        """
        Load the train dataset annotation using pandas
        
        :return train_label: pandas dataframe, whales id <-> files matching
        """
        print("Loading", self.dataset_path_train_label)
        return pd.read_csv(self.dataset_path_train_label)
    
    
    def create_folder_is_needed(self, folder_path):
        """
        Create a folder if it doesn't alreadt exist
        :param folder_path: str
        """
        if not os.path.isdir(folder_path):
            os.mkdir(folder_path)

    
    def split_in_classes_folders(self, root_classes_folder, pass_new_whale=True, remove_old=True, train_dev_ratio=0.2):
        """
        Split the dataset into classes folders
        :param root_classes_folder:
        :param pass_new_whale: boolean, set to true if not considering new_whale id
        
        OUTPUT:
            pictures sort by whale id into subfolders of the root_classes_folder
        """
        
        """if remove_old:
            print("Removing previous spliting")
            shutil.rmtree(root_classes_folder)"""
        
        sub_dataset_train_path = "/".join([root_classes_folder, "train"])
        sub_dataset_dev_path = "/".join([root_classes_folder, "dev"])
        
        
        # Create the folders needed
        created_folder_path_train_fake = "dataset/siamesedata/train/fake"
        created_folder_path_train_real = "dataset/siamesedata/train/real"
        created_folder_path_dev_fake = "dataset/siamesedata/dev/fake"
        created_folder_path_dev_real = "dataset/siamesedata/dev/real"
        
        self.create_folder_is_needed(created_folder_path_train_fake)
        self.create_folder_is_needed(created_folder_path_train_real)
        self.create_folder_is_needed(created_folder_path_dev_fake)
        self.create_folder_is_needed(created_folder_path_dev_real)
        
        # If passing new whales, should remove previous folder
        new_whale_folder_path = "/".join([root_classes_folder, "new_whale"])
        if pass_new_whale and os.path.isdir(new_whale_folder_path):
            print("Removing new_whale folder")
            shutil.rmtree(new_whale_folder_path)
        
        # Sorting the images
        files_number = len(self.dataset_train_label)
        print("Sorting", files_number, "images into", root_classes_folder)
        
        for index, row in self.dataset_train_label.iterrows():
            print("#" + str(index + 1) + "/" + str(files_number), end="\r")
            whale_file_name = row['Image']
            whale_id = row['Id']
            if pass_new_whale and whale_id == "new_whale":
                continue
            
            ## Choose if storing in train of dev dataset
            if len(glob.glob("/".join([sub_dataset_train_path, whale_id]))):
                if random.uniform(0, 1) >= 0.2:
                    # We store in train dataset
                    sub_dataset_path = sub_dataset_train_path
                else:
                    # We store in dev dataset
                    sub_dataset_path = sub_dataset_dev_path
            else:
                sub_dataset_path = sub_dataset_train_path
            
            self.create_folder_is_needed("/".join([sub_dataset_path, whale_id]))
            shutil.copy(
                "/".join([self.dataset_path_train, whale_file_name]), 
                "/".join([sub_dataset_path, whale_id, whale_file_name])
            )
        print("Done" + " "*20)
        
        ## Removing folder that doesn't have a dev equivalent
        for train_path in glob.glob(sub_dataset_train_path + "/*/"):
            in_train_id = train_path.split("/")[-2]
            equivalent_dev_id_path = "/".join([sub_dataset_dev_path, in_train_id])
            if not os.path.isdir(equivalent_dev_id_path):
                shutil.rmtree(train_path)
            
# Create the dataset object
#dataset = Dataset("dataset")
# Split the dataset into the train_classes folder
#dataset.split_in_classes_folders("dataset/train_classes")



class DataGenerator(object):
    """docstring for DataGenerator"""
    def __init__(self, batch_sz, train):
        # the data, shuffled and split between train and test sets
        #dataset = Dataset("dataset")
        #sub_dataset_train_path = "/".join([root_classes_folder, "train"])
        #sub_dataset_dev_path = "/".join([root_classes_folder, "dev"])
        
        # Create the folders needed
        created_folder_path_train_fake = "dataset/siamesedata/train/fake"
        created_folder_path_train_real = "dataset/siamesedata/train/real"
        created_folder_path_dev_fake = "dataset/siamesedata/dev/fake"
        created_folder_path_dev_real = "dataset/siamesedata/dev/real"
        
        self.create_folder_is_needed(created_folder_path_train_fake)
        self.create_folder_is_needed(created_folder_path_train_real)
        self.create_folder_is_needed(created_folder_path_dev_fake)
        self.create_folder_is_needed(created_folder_path_dev_real)
        
        
        #Load the data 
        data_train = pd.read_csv("./dataset/train.csv")
        fig = "train"
        
        real_data, fake_data = load_fake_real_data(data_train)
        
        total_data = np.concatenate()
        
        
            
        
        
        
        list_X_train = []
        for fig in data_train['Image']:
            img = image.load_img("./dataset/"+dataset+"/"+fig, target_size=(200, 100, 3))
            x = image.img_to_array(img)
            list_train.append([x])
        X_train = np.vstack(list_X_train)
            
        y_train, label_encoder = prepare_labels(data_train['Id'])
        
            
            
        
        X_train, y_train
        , (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255



        self.batch_sz = batch_sz
        self.samples_per_train  = (self.tr_pairs.shape[0]/self.batch_sz)*self.batch_sz
        self.samples_per_val    = (self.te_pairs.shape[0]/self.batch_sz)*self.batch_sz


        self.cur_train_index=0
        self.cur_val_index=0
    def load_fake_real_data(train_data):
        print("Loading...")
        train_data.set_index('Id', inplace=True)
        train_data.sort_index(inplace=True)
        fake_data = []
        real_data = []
        #k=0

        for sub_index in set(train_data.index):
            if sub_index!="new_whale":
                #print(k)
                #print(sub_index)
                #k+=1
                images_same_class = list(set(train_data.loc[train_data.index==sub_index].Image))
                images_different_class = list(set(train_data.loc[train_data.index!=sub_index].Image))

                for i in range(len(images_same_class)):
                    for j in range(i, len(images_same_class)):
                        real_data.append(np.array([images_same_class[i], images_same_class[j],1]))
                        fake_data.append(np.array([images_same_class[i], images_different_class[np.random.randint(len(images_different_class))],0]))
                        fake_data.append(np.array([images_same_class[i], images_different_class[np.random.randint(len(images_different_class))],0]))
        
        fake_data = np.vstack(fake_data)
        real_data = np.vstack(real_data)
        print("Fake and Real data generated for siamese training.")
        return real_data, fake_data

    def prepare_labels(y):
        values = np.array(y)
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(values)
        # print(integer_encoded)

        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        # print(onehot_encoded)

        y = onehot_encoded
        # print(y.shape)
        return y, label_encoder

    def create_pairs(self, x, digit_indices):
        '''Positive and negative pair creation.
        Alternates between positive and negative pairs.
        '''
        pairs = []
        labels = []
        n = min([len(digit_indices[d]) for d in range(10)]) - 1
        for d in range(10):
            for i in range(n):
                z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
                pairs += [[x[z1], x[z2]]]
                inc = random.randrange(1, 10)
                dn = (d + inc) % 10
                z1, z2 = digit_indices[d][i], digit_indices[dn][i]
                pairs += [[x[z1], x[z2]]]
                labels += [1, 0]
        return np.array(pairs), np.array(labels)

    def next_train(self):
        while 1:
            self.cur_train_index += self.batch_sz
            if self.cur_train_index >= self.samples_per_train:
                self.cur_train_index=0
            yield ([    self.tr_pairs_0[self.cur_train_index:self.cur_train_index+self.batch_sz], 
                        self.tr_pairs_1[self.cur_train_index:self.cur_train_index+self.batch_sz]
                    ],
                    self.tr_y[self.cur_train_index:self.cur_train_index+self.batch_sz]
                )

    def next_val(self):
        while 1:
            self.cur_val_index += self.batch_sz
            if self.cur_val_index >= self.samples_per_val:
                self.cur_val_index=0
            yield ([    self.te_pairs_0[self.cur_val_index:self.cur_val_index+self.batch_sz], 
                        self.te_pairs_1[self.cur_val_index:self.cur_val_index+self.batch_sz]
                    ],
                    self.te_y[self.cur_val_index:self.cur_val_index+self.batch_sz]
                )

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


input_dim = 784
nb_epoch = 20
batch_size=128

datagen = DataGenerator(batch_size)

# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input=[input_a, input_b], output=distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit_generator(generator=datagen.next_train(), samples_per_epoch=datagen.samples_per_train, nb_epoch=nb_epoch, validation_data=datagen.next_val(), nb_val_samples=datagen.samples_per_val)

Loading dataset/train.csv


FileNotFoundError: [Errno 2] File b'dataset/train.csv' does not exist: b'dataset/train.csv'